# Pratique o Fluxo de Desenvolvimento FPGA

##### Sections
- [oneAPI with Intel® FPGAs](#oneAPI-with-Intel®-FPGAs)
- [Stage 1: Emulation](#Stage-1:-Emulation)
- [Stage 2: Optimization Report Generation](#Stage-2:-Optimization-Report-Generation)
- [How to Use the Terminal within Jupyter Lab](#How-to-Use-the-Terminal-within-Jupyter-Lab)
- [Emulation and Optimization Report Generation with Included Tutorials](#Emulation-and-Optimization-Report-Generation-with-Included-Tutorials)
- [References to Learn More](#References-to-Learn-More)

## Objetivos de aprendizado

* Compreenda o fluxo de desenvolvimento para Intel® FPGAs com o Intel® oneAPI Toolkits
* Pratique o uso do fluxo com um código simples
* Pratique o uso do fluxo com os tutoriais FPGA incluídos no Intel oneAPI Base Toolkit

***
# oneAPI com FPGAs Intel® 

O fluxo de desenvolvimento para FPGAs Intel com oneAPI envolve vários estágios. O objetivo dessas etapas é para que você possa:
* Garantir a funcionalidade do seu código (obter as respostas corretas do seu cálculo)
* Certificar-se de que o hardware personalizado desenvolvido para implementar seu código tenha um desempenho ideal

Sem ter que aguradar a longa compilação para um executável FPGA completo a cada vez.

O fluxo é representado no diagrama abaixo.

Neste laboratório, praticaremos os dois primeiros estágios do fluxo - emular seu código para garantir que ele funcione e gerar um relatório de otimização para ver como está otimizado o hardware gerado a partir de seu código. (Um laboratório subsequente dará a você a prática de trabalhar com o relatório de otimização.)

Há um tutorial para casa se você quiser fazer uma compilação completa do FPGA e executá-lo em um FPGA.

<img src="Assets/fpga_flow.png">

***
# Estágio 1: Emulação

O primeiro estágio de desenvolvimento para FPGAs com oneAPI é a __emulação__. O objetivo da emulação é ter certeza de que seu código é __funcional__, ou em outras palavras, que você vá __obter as respostas corretas de seus cálculos__.

O tempo de compilação para este estágio será muito rápido, geralmente segundos.

Esse rápido tempo de compilação permite que você itere por esse estágio muitas vezes, até que seu código esteja funcionalmente correto.

__Agora vamos tentar!__

O código a seguir implementa uma soma cumulativa simples em uma matriz de valores.

O código é bem comentado, portanto, dê uma olhada rápida agora para obter uma visão geral do que ele está fazendo. (Além disso - tenha em mente que este é um exemplo simples. Não valeria a pena usar um acelerador para somar 100 inteiros!)

Usaremos esse código simples para aprender as etapas do fluxo de desenvolvimento para FPGAs com a oneAPI.

__Depois de terminar de examinar o código, clique em ▶ para salvar o código em um arquivo .__

In [ ]:
%%writefile lab/simple_for_fpga.cpp
//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <CL/sycl.hpp>
#include <CL/sycl/intel/fpga_extensions.hpp>
using namespace sycl;
static const int N = 100;
int main(){
    
  //# Cria uma matriz de 100 números incrementais
  //# A soma deve ser 5050
  int summands[100];
  for (int i=0;i<100;i++) summands[i]=i+1;
    
  //# Cria uma variável para guardar a soma
  int sum = 0;
    
  //# Um flag -D definirá qual dispositivo escolheremos
  #if defined(FPGA_EMULATOR)
    intel::fpga_emulator_selector device_selector;
  #else
    intel::fpga_selector device_selector;
  #endif

  //# Buffers são usados ​​para compartilhar dados entre o host e o FPGA
  buffer<int, 1> buffer_summands(summands, 100);
  buffer<int, 1> buffer_sum(&sum, 1);

  //# define a fila que tem dispositivo padrão associado para descarregamento
  //# A fila é usada pelo host para iniciar o código no FPGA
  queue q(device_selector);
    
  //# Envia os valores para o FPGA ou o emulador FPGA para calcular a soma
  //# Você pode pensar no manipulador como um intermediário para tudo 
  //# o que precisa acontecer entre o host e o FPGA
  q.submit([&](handler &h) {
    //# O FPGA precisa ter acesso aos buffers configurados anteriormente
    //# O acesso é definido em termos de acesso do lado do FPGA
    auto acc_summands = buffer_summands.get_access<access::mode::read>(h);
    auto acc_sum = buffer_sum.get_access<access::mode::write>(h);
      
    //# Este é o código que é executado no FPGA
    //# Isso geralmente é referido como um kernel
    //# Se você quisesse tornar simple_sum uma função, você poderia,
    //# e os Tutoriais FPGA são escritos desta maneira
    h.single_task<class simple_sum>([=]() {
      //# Kernel para adicionar coisas usando FPGA ou emulador FPGA
      //# O código aqui se torna hardware
      int kernel_sum = 0;
      for (int i=0;i<100;i++) kernel_sum = kernel_sum + acc_summands[i];
      acc_sum[0] = kernel_sum;
    });
  }).wait();

  //# Print Output
  std::cout << "The calculation is finished. The sum is ";
  std::cout << sum;
  std::cout << "." << std::endl;

  return 0;
}

__Agora você irá compilar o código para o emulador FPGA.__

Lembre-se da aula, o comando para fazer isso é mostrado abaixo.

<img src="Assets/emulator_command.png">

O comando que usaremos também adiciona uma opção -o para definir o nome do arquivo de saída.

__Agora ative a seção de código abaixo (você verá uma barra azul ao lado da seção) e clique em ▶.__ Isso irá compilar o código em um executável para o emulador FPGA e, em seguida, executar o código emulado. Você verá a saída das instruções `std::cout` que estão no código.

In [ ]:
! echo "##" $(whoami) is working oneAPI_Toolkits_with_FPGAs FPGA_dev_flow lab step 1
! dpcpp -fintelfpga lab/simple_for_fpga.cpp -DFPGA_EMULATOR -o bin/simple_for_fpga.emu
! bin/simple_for_fpga.emu

__Você deve ter visto uma saída parecida com a abaixo:__

`The calculation is finished. The sum is 5050.`

__É sempre útil ver o que acontece quando as coisas não ocorrem perfeitamente.__

Volte para o código que você acabou de executar e introduza um erro de sintaxe (ou alguns). Em seguida, clique em ▶ para a seção do Notebook com o código e ▶ para a seção do Notebook para compilar e executar o código com o emulador FPGA.

__Você pode ver como é rápido e fácil emular seu código! __

Isso foi rápido, o software compila como a maioria dos desenvolvedores de software estão acostumados! Essa rápida compilação e execução são o motivo pelo qual você permanece neste estágio até que seu código esteja funcional. (ou seja, você está obtendo as respostas corretas do seu código!)

***
## Estágio 2: Geração de relatório de otimização

Nesta próxima seção do laboratório, você irá compilar o _kernel_ usando diferentes opções de linha de comando com o `dpcpp` para criar um relatório de otimização. Você também usará a interface do Jupyter para navegar e abrir um arquivo, então isso será explicado a você.

Como a última parte do exercício disse para você ver o que acontece quando você introduz erros de sintaxe no código, vamos começar do zero escrevendo o código de exemplo em um arquivo novamente. Ative a seção de código do bloco de notas abaixo e pressione ▶ para reescrever o exemplo de código simples em um arquivo.

In [ ]:
%%writefile lab/simple_for_fpga.cpp
//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <CL/sycl.hpp>
#include <CL/sycl/intel/fpga_extensions.hpp>
using namespace sycl;
static const int N = 100;
int main(){
    
  //# Cria uma matriz de 100 números incrementais
  //# A soma deve ser 5050
  int summands[100];
  for (int i=0;i<100;i++) summands[i]=i+1;
    
  //# Cria uma variável para guardar a soma
  int sum = 0;
    
  //# Um flag -D definirá qual dispositivo escolheremos
  #if defined(FPGA_EMULATOR)
    intel::fpga_emulator_selector device_selector;
  #else
    intel::fpga_selector device_selector;
  #endif

  //# Buffers são usados ​​para compartilhar dados entre o host e o FPGA
  buffer<int, 1> buffer_summands(summands, 100);
  buffer<int, 1> buffer_sum(&sum, 1);

  //# define a fila que tem dispositivo padrão associado para descarregamento
  //# A fila é usada pelo host para iniciar o código no FPGA
  queue q(device_selector);
    
  //# Envia os valores para o FPGA ou o emulador FPGA para calcular a soma
  //# Você pode pensar no manipulador como um intermediário para tudo 
  //# o que precisa acontecer entre o host e o FPGA
  q.submit([&](handler &h) {
    //# O FPGA precisa ter acesso aos buffers configurados anteriormente
    //# O acesso é definido em termos de acesso do lado do FPGA
    auto acc_summands = buffer_summands.get_access<access::mode::read>(h);
    auto acc_sum = buffer_sum.get_access<access::mode::write>(h);
      
    //# Este é o código que é executado no FPGA
    //# Isso geralmente é referido como um kernel
    //# Se você quisesse tornar simple_sum uma função, você poderia,
    //# e os Tutoriais FPGA são escritos desta maneira
    h.single_task<class simple_sum>([=]() {
      //# Kernel para adicionar coisas usando FPGA ou emulador FPGA
      //# O código aqui se torna hardware
      int kernel_sum = 0;
      for (int i=0;i<100;i++) kernel_sum = kernel_sum + acc_summands[i];
      acc_sum[0] = kernel_sum;
    });
  }).wait();

  //# Print Output
  std::cout << "The calculation is finished. The sum is ";
  std::cout << sum;
  std::cout << "." << std::endl;

  return 0;
}

__Agora você vai compilar o código para gerar um relatório de otimização.__

Lembre-se da aula, os comandos para fazer isso em duas etapas são mostrados abaixo. (Estamos usando o método de duas etapas, pois há um problema atual que mostra o código-fonte no relatório com o método de uma etapa.)

<img src="Assets/report_command.png">

O comando a seguir também inclui uma opção -o para que o arquivo de saída possa receber um nome explícito.

__Ative a seção de código abaixo e pressione ▶ para compilar o código e gerar um relatório de otimização.__
__Esta compilação demora em torno de 2 minutos.__

In [ ]:
! echo "##" $(whoami) is working oneAPI_Toolkits_with_FPGAs FPGA_dev_flow lab step 2
! dpcpp -fintelfpga lab/simple_for_fpga.cpp -c -o bin/simple_for_fpga.o
! dpcpp -fintelfpga bin/simple_for_fpga.o -fsycl-link -Xshardware -o bin/simple_for_fpga.a
! echo "The compile is finished."

__Quando você vir `The compile is finished.` acima, um arquivo de relatório de otimização terá sido gerado para o código. Você verá um aviso se compilar mais de uma vez. O aviso pode ser ignorado.__

Agora, vamos examinar esse arquivo de relatório.

No ambiente do Jupyter, você verá os arquivos a serem navegados no lado direito. Navegue até o diretório `fpga_dev_flow/bin/simple_for_fpga.prj/reports/` (clique duas vezes nos diretórios para entrar neles).

O lado esquerdo da tela deve ser semelhante à imagem abaixo:

<img src="Assets/browse_to_report.png">

__Clique duas vezes em report.html.__ O relatório será aberto como outra guia ao lado da guia do bloco de notas no Jupyter, conforme mostrado abaixo.

__Provavelmente, você precisará clicar em "Trust HTML" para que o relatório seja totalmente aberto.__

<img src="Assets/report_initial_open.png">

__Uma vez que você clicar em "Trust HTML", será semelhante à captura de tela abaixo .__

<img src="Assets/trusted_report.png">

__Caso você tenha dificuldades para ver o relatório diretamente na DevCloud, você pode usar o comando abaixo para copiá-lo para o seu computador:__

`scp -r devcloud:~/sycl-fpga-wscad-2020/labs/lab2/bin/simple_for_fpga.prj/reports .`

Agora você aprendeu como usar os dois primeiros estágios do fluxo de desenvolvimento para FPGAs com oneAPI! Você passará a maior parte do tempo de desenvolvimento nesses dois estágios. Nas próximas seções do laboratório você aprenderá a trabalhar com os tutoriais integrados do FPGA e os projetos de exemplo.

***
## Como usar o terminal dentro do Jupyter

Na próxima sessão de laboratório, você trabalhará no terminal que faz parte do Jupyter. Trabalhar com o terminal ao executar o Jupyter no DevCloud é como trabalhar em um terminal dentro de um ambiente de desenvolvimento Linux. Uma vez aberto, os comandos serão os mesmos comandos do Linux aos quais você está acostumado.

__Para abrir um terminal dentro do ambiente Jupyter, clique primeiro no "+" próximo ao canto superior esquerdo do ambiente Jupyter em seu navegador.__ O "+" em que você precisa clicar tem uma caixa vermelha desenhada ao redor na captura de tela abaixo.

<img src="Assets/button_for_launcher.png">

Depois de clicar em "+", uma guia do iniciador aparecerá. __Clique no ícone do terminal no inicializador, conforme mostrado abaixo .__

<img src="Assets/button_for_terminal.png">

__Depois disso, uma nova guia, que é um terminal, aparecerá na área de trabalho do Jupyter.__ Você receberá um prompt. Para começar, você será colocado em seu diretório doméstico. A guia do terminal com o prompt é mostrada abaixo.

<img src="Assets/terminal_with_prompt.png">

A próxima seção fornecerá instruções para executar nesta janela de terminal. Observe que você pode selecionar o comando que vai executar, pressionar Ctrl-C para copiá-lo e, se desejar, pressionar Ctrl-V para colá-lo no prompt.

***
## Emulação e geração de relatório de otimização com tutoriais incluídos

O oneAPI Base Toolkit inclui muitos tutoriais e exemplos para aprender mais sobre como escrever código otimizado para FPGAs. Nesta seção do laboratório você aprenderá como gerar e compilar esses exemplos e tutoriais.

Todos os exemplos e tutoriais usam o processo de compilação CMake. Se você estiver familiarizado com os processos de compilação CMake ou Make, as instruções nesta seção serão familiares para você.

Para começar, vá para o prompt do terminal que você abriu na última seção do laboratório.

Navegue até o diretório de trabalho do laboratório. Se você colocou os arquivos de laboratório em seu diretório doméstico, o comando será como mostrado abaixo.

~~~ bash
$ cd ~/sycl-fpga-wscad-2020/labs
~~~

O comando para acessar os tutoriais FPGA incorporados ao kit de ferramentas é mostrado abaixo. Execute este comando no prompt agora.

~~~ bash
$ oneapi-cli
~~~

Depois de digitar este comando no terminal, o terminal deve se parecer com a imagem abaixo.

<img src="Assets/terminal_oneapi_cli.png">

Selecione __(1) Create a project__

Em seguida, selecione __(1) cpp__

A tela do seu terminal deve ser semelhante à imagem abaixo.

<img src="Assets/tutorial_choices.png">

Como você pode ver, existem muitas opções para Tutoriais FPGA e Projetos de Exemplo FPGA. Os tutoriais fornecem código de exemplo para ajudá-lo a aprender mais sobre tópicos individuais. A maioria dos tópicos cobertos no guia de programação e nos guias de otimização são cobertos.

Role para baixo até __FPGA Tutorial: Loop Unroll__ e pressione Enter. No campo Directory, digite __tutorial__, conforme mostrado abaixo.

<img src="Assets/tutorial_gen.png">

Em seguida, pressione `Enter` até chegar no __Create__, confirmando com mais um `Enter`. Então, pressione `Enter` no __Quit__.

Agora você deve estar de volta ao prompt do terminal.

No prompt, altere seu diretório para entrar no tutorial gerado.

~~~ bash
$ cd tutorial/loop_unroll
~~~

Dê uma olhada nos arquivos gerados digitando `ls` no prompt.

~~~ bash
$ ls
~~~

Visualize o arquivo README.md se desejar, abrindo-o no terminal usando `vi` (se você sourver sair depois ;-) ou `less`, ou navegando até ele e clicando nele usando o navegador de arquivos embutido à esquerda no Jupyter.

As próximas instruções também podem ser encontradas no arquivo README.md, se você preferir trabalhar a partir dele. Este laboratório trabalhará nas seções "Compile and run for emulation" e "Generate HTML optimization reports".

Primeiro, crie um diretório de construção e prepare-o para executar o processo de construção do CMake, executando os seguintes comandos (eles assumem que você inicia a partir do diretório `tutorial/loop_unroll` criado quando você gerou o tutorial.

```bash
$ mkdir build
$ cd build
$ cmake ..
```

Em seguida, compile o tutorial para emulação e execute o executável de emulação resultante digitando os seguintes comandos:

~~~ bash
$ make fpga_emu
$ ./loop_unroll.fpga_emu
~~~

Depois de executar esses comandos, a tela do seu terminal deve ser semelhante à imagem abaixo. __Note que, como o comando está sendo executado como um trecho de código emulado, os números de desempenho não refletem o desempenho do FPGA. Você teria que fazer a compilação completa para o FPGA e executar no FPGA para esses números.__

<img src="Assets/tutorial_emu_output.png">

__Na sequência, você compilará para gerar um relatório de otimização .__

No prompt, digite o seguinte comando para gerar um relatório de otimização para o exemplo do tutorial.

~~~ bash
$ make report
~~~

A saída para o seu terminal deve ser semelhante à imagem abaixo.

<img src="Assets/tutorial_report_build.png">

Agora localize o arquivo `report.html` no navegador de arquivos à esquerda na interface do Jupyter. A localização do arquivo será em:

~~~ bash
~/sycl-fpga-wscad-2020/labs/tutorial/loop_unroll/build/loop_unroll_report.prj/reports/
~~~

Abra o arquivo `report.html` dentro do Jupyter clicando duas vezes. __Lembrete: Pode ser necessário clicar em "Trust HTML" ou transferir o relatório para ver localmente__.

O relatório deve ser semelhante à captura de tela abaixo.

<img src="Assets/unroll_example_report.png">

Como um lembrete, como a compilação para um executável FPGA completo envolve um tempo de compilação muito longo e o Guia de Otimização do FPGA da Intel® oneAPI DPC++, há nós limitados na DevCloud disponíveis para isso, então não faremos essa etapa. Existe um tutorial se você quiser tentar depois da aula.

__Agora você concluiu o laboratório!__

Se você tiver tempo extra, pode fazer o seguinte:
* Interaja com seus colegas se o bate-papo pelo Discord estiver habilitado para sua sessão de aula.
* Leia as referências abaixo para saber mais.
* Gere mais exemplos de tutoriais (isso é especialmente bom se você também estiver lendo o Intel® oneAPI DPC++ FPGA Optimization Guide).
* Descanse um pouco antes de começar a próxima seção da aula.

***
## Referências para aprender mais

Consulte os seguintes recursos para saber mais. Isso é ótimo se você tiver tempo extra durante o laboratório!

#### Documentação específica para FPGAs

* [Website hub for using FPGAs with oneAPI](https://software.intel.com/content/www/us/en/develop/tools/oneapi/components/fpga.html)
* [Intel® oneAPI Programming Guide](https://software.intel.com/content/www/us/en/develop/download/intel-oneapi-programming-guide.html)
* [Intel® oneAPI DPC++ FPGA Optimization Guide](https://software.intel.com/content/www/us/en/develop/download/oneapi-fpga-optimization-guide.html)
* [FPGA Tutorials GitHub](https://github.com/intel/BaseKit-code-samples/tree/master/FPGATutorials)

#### Documentação Intel® oneAPI Toolkit 
* [Intel® oneAPI main page](https://software.intel.com/oneapi "oneAPI main page")
* [Intel® oneAPI programming guide](https://software.intel.com/sites/default/files/oneAPIProgrammingGuide_3.pdf "oneAPI programming guide")
* [Intel® DevCloud Signup](https://software.intel.com/en-us/devcloud/oneapi "Intel DevCloud") Cadastre-se aqui se você não possui uma conta.
* [Intel® DevCloud Connect](https://devcloud.intel.com/datacenter/connect) Faça login no DevCloud aqui.
* [Get Started with oneAPI for Linux*](https://software.intel.com/en-us/get-started-with-intel-oneapi-linux)
* [Get Started with oneAPI for Windows*](https://software.intel.com/en-us/get-started-with-intel-oneapi-windows)
* [Intel® oneAPI Code Samples](https://software.intel.com/en-us/articles/code-samples-for-intel-oneapibeta-toolkits)
* [oneAPI Specification elements](https://www.oneapi.com/spec/)

#### SYCL 
* [SYCL* Specification (for version 1.2.1)](https://www.khronos.org/registry/SYCL/specs/sycl-1.2.1.pdf)
* [SYCL* Specification (for version 2020)](https://www.khronos.org/registry/SYCL/specs/sycl-2020-provisional.pdf)

#### C++ Moderno
* [CPPReference](https://en.cppreference.com/w/)
* [CPlusPlus](http://www.cplusplus.com/)